In [142]:
import geopandas as gpd
import pandas as pd

## Open State data

In [143]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

## Get Poverty Percents

In [169]:
acs_internet = pd.read_csv(
    "data/ACSST5Y2023.S2801-2025-04-30T214312.csv", thousands=","
)

In [170]:
acs_internet["Label (Grouping)"] = acs_internet["Label (Grouping)"].str.strip()

In [171]:
acs_percent_df = pd.DataFrame(
    acs_internet[
        acs_internet["Label (Grouping)"] == "Without an Internet subscription"
    ].iloc[0]
).reset_index(drop=False)

In [172]:
acs_percent_df = acs_percent_df.rename(columns={"index": "STATE", 20: "PERCENT"})
acs_percent_df = acs_percent_df.iloc[1:]

In [173]:
total_df = acs_percent_df[acs_percent_df["STATE"].str.contains("Total")]
percent_df = acs_percent_df[acs_percent_df["STATE"].str.contains("Percent")]

In [174]:
total_df["NAME"] = total_df["STATE"].str.split("!!").str[0]
total_df["PERCENT"] = total_df["PERCENT"].astype(int)
total_df = total_df.rename(columns={"PERCENT": "TOTAL"})
percent_df["NAME"] = percent_df["STATE"].str.split("!!").str[0]
percent_df["PERCENT"] = percent_df["PERCENT"].str[:-1].astype(float)
total_df = total_df.drop("STATE", axis=1)
percent_df = percent_df.drop("STATE", axis=1)

/tmp/ipykernel_18430/842281958.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_df['NAME'] = total_df['STATE'].str.split("!!").str[0]
/tmp/ipykernel_18430/842281958.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_df['PERCENT'] = total_df['PERCENT'].astype(int)
/tmp/ipykernel_18430/842281958.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

In [175]:
internet_df = percent_df.merge(total_df, on="NAME", how="inner")

## Merge Data

In [177]:
internet_percents_gdf = states_df.merge(internet_df, on="NAME", how="inner").dropna()

In [178]:
internet_percents_gdf = internet_percents_gdf[["NAME", "geometry", "PERCENT", "TOTAL"]]

In [179]:
internet_percents_gdf = internet_percents_gdf.to_crs(9311)
internet_percents_gdf.to_file("data/internet.gpkg")